In [1]:
#============================================================================
# import packages
import cv2
import numpy as np
import imutils
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
#============================================================================
# load data
src = cv2.imread('./data/input image.jpg', cv2.IMREAD_GRAYSCALE)
tmp = cv2.imread('./data/template.jpg', cv2.IMREAD_GRAYSCALE)
dst  = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)  # 출력 표시 영상

#============================================================================

blurred = cv2.GaussianBlur(dst,(3,3), 0.0)
canny = cv2.Canny(blurred, 80 , 250) #30 250
# canny = imutils.auto_canny(blurred)

#================================================================================

ret, labels, stats, centroids = cv2.connectedComponentsWithStats(canny)
print("ret= ", ret)

maxX = maxY = 0
maxWidth = maxHeight = 0
size = 0
maxPoint = 0

for i in range(1, ret):
    x, y, width, height, area = stats[i]
    
    if(area > size):
        maxX = x
        maxY = y
        maxWidth = width
        maxHeight = height
        size = area
        maxPoint = i
    #바운딩 박스를 만들어준다.
   # cv2.rectangle(dst, (x,y), (x+width, y + height), (0,0,255), 2)

print(stats[maxPoint][0] , stats[maxPoint][1], stats[maxPoint][2], stats[maxPoint][3])

dst = dst - 1

dst[stats[maxPoint][1] : stats[maxPoint][1] + stats[maxPoint][3],stats[maxPoint][0] : stats[maxPoint][0]+stats[maxPoint][2] ] = 255
maxPoint = maxPoint + 2
dst[stats[maxPoint][1] : stats[maxPoint][1] + stats[maxPoint][3],stats[maxPoint][0] : stats[maxPoint][0]+stats[maxPoint][2] ] = 255



# cv2.rectangle(dst, (stats[maxPoint][0],stats[maxPoint][1]), (stats[maxPoint][0]+stats[maxPoint][2], 
#                                                              stats[maxPoint][1] + stats[maxPoint][3]), (0,0,255), 2)
# cv2.rectangle(dst, (maxX,maxY), (maxX+maxWidth, maxY + maxHeight), (0,0,255), 2)

# print(tmp.shape[1],tmp.shape[0] ,maxHeight)
# # tmp = cv2.resize(tmp, dsize=(500, maxHeight))
# # dst  = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
_, dst = cv2.threshold(dst, 254, 255, cv2.THRESH_BINARY)

kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3,3))
dst = cv2.erode(dst,kernel, iterations = 1)
dst = cv2.dilate(dst, kernel, iterations = 1)

test = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
#==================================================================================

cnts, _ = cv2.findContours(test, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(cnts)

maxX = maxY = 0
maxWidth = maxHeight = 0
size = 0

#loop over the contours
for c in cnts:
    #fit a bounding box to the contour
    (x, y, w, h) = cv2.boundingRect(c)
    if(w*h > size):
        size = w * h
        maxWidth = w
        maxHeight = h
    cv2.rectangle(dst, (x, y), (x + w, y + h), (0,255,0), 2)
    
print(size , maxWidth, maxHeight)

tmp = cv2.resize(tmp, dsize=(maxWidth+5, maxHeight))

r1 = cv2.matchTemplate(src, tmp, cv2.TM_SQDIFF_NORMED)
minVal , _ , minLoc, _ = cv2.minMaxLoc(r1)
print("TM_SQDIFF_NORMED: ", minVal, minLoc)
w, h = tmp.shape[:2]

src = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.rectangle(src, minLoc, (minLoc[0]+h, minLoc[1]+w), (0,0,255), 2)

cv2.imshow("Blurred", blurred)
cv2.imshow("canny", canny)
cv2.imshow("dst", dst)
cv2.imshow("tmp", tmp)
cv2.imshow("src", src)
cv2.waitKey()
cv2.destroyAllWindows()

ret=  34
257 159 129 150
[array([[[257, 159]],

       [[257, 308]],

       [[385, 308]],

       [[385, 277]],

       [[386, 276]],

       [[433, 276]],

       [[433, 174]],

       [[386, 174]],

       [[385, 173]],

       [[385, 159]]], dtype=int32)]
26550 177 150
TM_SQDIFF_NORMED:  0.020652467384934425 (251, 159)
